In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from sklearn import metrics
import catboost as cb
from sklearn.ensemble import RandomForestRegressor

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error

from sklearn.model_selection import train_test_split

from sklearn.impute import SimpleImputer

from mlxtend.preprocessing import minmax_scaling
# for Box-Cox Transformation
from scipy import stats
import xgboost as xgb
from sklearn.metrics import roc_auc_score

from sklearn import preprocessing
import lightgbm as lgbm
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
import warnings 
warnings.filterwarnings('ignore')

In [43]:
def preprocess(train,test):
    #catogiry
    
    cat_columns = train.select_dtypes(include=['object'])
    cat_columns = cat_columns.fillna('East Asia & Pacific (excluding high income)').replace({'Taiwan, China':'Taiwan'})
    # delet missing value < 50%
    train.pop('Q7')
    train.pop('Q23')
    train.pop('Q3')
    train.pop('Q2')
    
    test.pop('Q7')
    test.pop('Q23')
    test.pop('Q3')
    test.pop('Q2')
    
    train.loc[:,['country','region']]  = cat_columns[['country','region']]
    test.loc[:,['country','region']]  = cat_columns[['country','region']]
    
    #normalizaiont population
#     original_data = train.population
#     normalized_data = stats.boxcox(original_data)
#     train.population = normalized_data[0]
    
#     original_data = test.population
#     normalized_data = stats.boxcox(original_data)
#     test.population = normalized_data[0]
    
    # scaled 
    scaled_data = minmax_scaling(train.population, columns=[0])
    train.population = scaled_data
    
    scaled_datat = minmax_scaling(test.population, columns=[0])
    test.population = scaled_datat

################### Age ######################################
#     original_data = train.age
#     normalized_data = stats.boxcox(original_data)
#     train.age = normalized_data[0]
    
#     original_data = test.age
#     normalized_data = stats.boxcox(original_data)
#     test.age = normalized_data[0]
    
    scaled_data = minmax_scaling(train.population, columns=[0])
    train.population = scaled_data
    
    scaled_datat = minmax_scaling(test.population, columns=[0])
    test.population = scaled_datat

# Fill in the median below: imputation
    smimpute = SimpleImputer(strategy='mean')
    numeric_data = train.select_dtypes(exclude=['object'])
    imputed_with_median = pd.DataFrame(smimpute.fit_transform(numeric_data),columns=numeric_data.columns)
    train.loc[:,numeric_data.columns] = imputed_with_median

    
    numeric_datat = test.select_dtypes(exclude=['object'])
    imputed_with_mediant = pd.DataFrame(smimpute.fit_transform(numeric_datat),columns=numeric_datat.columns)
    test.loc[:,numeric_datat.columns] = imputed_with_mediant
    
    #Label Incode
    le = preprocessing.LabelEncoder()
    le.fit(train.country)
    train.country = le.transform(train.country)
    le.fit(test.country)
    test.country = le.transform(test.country)
    
    #One Hot Encoding
    # Apply one-hot encoder to each column with categorical data
    OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
    OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(train[['region']]))
    OH_cols_train.index = train.index
    train = train.drop(['region'],axis=1)
    train = pd.concat([train, OH_cols_train], axis=1)
    
    OH_cols_test = pd.DataFrame(OH_encoder.fit_transform(test[['region']]))
    OH_cols_test.index = test.index
    test = test.drop(['region'],axis=1)
    test = pd.concat([test, OH_cols_test], axis=1)
    
    
    
    return train,test

In [32]:
df = pd.read_csv('/kaggle/input/financialresilience1234/Train.csv')
submission = pd.read_csv('/kaggle/input/financialresilience1234/SampleSubmission.csv')
test = pd.read_csv('/kaggle/input/financialresilience1234/Test.csv')
variable = pd.read_csv('/kaggle/input/financialresilience1234/VariableDefinitions.csv')

In [33]:
train,test = preprocess(df.copy(),test.copy())

In [34]:
X = train.drop(['target','ID'],axis=1)
y = train.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [26]:
train

,ID,country,population,age,Q1,Q4,Q5,Q6,Q8,Q9,...,Q28,target,0,1,2,3,4,5,6,7
0,ID_0000020532,89,0.022196,66.0,2.0,1.618835,1.556202,2.0,1.220485,1.202731,...,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,ID_0000075765,71,0.003718,44.0,2.0,2.000000,2.000000,2.0,1.000000,1.000000,...,1.000000,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,ID_0000085491,89,0.022196,29.0,1.0,2.000000,2.000000,2.0,1.000000,1.000000,...,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,ID_0000144505,21,0.026552,44.0,1.0,1.000000,1.000000,2.0,1.000000,1.000000,...,1.125586,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,ID_0000214184,85,0.081664,36.0,2.0,1.618835,1.556202,2.0,1.220485,1.202731,...,1.000000,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104862,ID_9999251978,54,0.837940,48.0,2.0,1.618835,1.556202,2.0,1.220485,1.202731,...,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
104863,ID_9999305731,52,0.005421,16.0,1.0,2.000000,2.000000,2.0,1.000000,1.000000,...,1.125586,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
104864,ID_9999406850,84,0.000574,53.0,1.0,2.000000,2.000000,2.0,2.000000,1.000000,...,1.125586,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
104865,ID_9999627239,45,0.063013,50.0,2.0,1.618835,1.556202,2.0,1.220485,1.202731,...,1.125586,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [27]:
cross_valid_scores = {}

In [45]:
%%time
parameters = {
    'max_depth' :[3,6,8,12],
#     'n_estimators': [100,200,300,400],
    'learning_rate': [0.1, 0.3],
    'num_leaves': [8, 16, 32, 64, 128]
}

model_lgbm = lgbm.LGBMClassifier(
     device =  "gpu",
    random_state=42,
    n_estimators = 400
   
)

model_lgbm = GridSearchCV(
    model_lgbm, 
    parameters, 
    cv=5,
    scoring='roc_auc',
)

model_lgbm.fit(
    X_train, 
    y_train
)
print('-----')
print(f'Best parameters {model_lgbm.best_params_}')
print(
    f'Mean cross-validated accuracy score of the best_estimator: ' + 
    f'{model_lgbm.best_score_:.3f}'
)
cross_valid_scores['lightgbm'] = model_lgbm.best_score_
print('-----')

-----
Best parameters {'learning_rate': 0.1, 'max_depth': 6, 'num_leaves': 16}
Mean cross-validated accuracy score of the best_estimator: 0.799
-----
CPU times: user 18min 40s, sys: 50.5 s, total: 19min 30s
Wall time: 10min 32s


In [41]:
def create_submission(model, test, model_name):
    y_pred_test = model.predict_proba(test.drop(['ID'],axis=1))[:, 1]
    sub = pd.DataFrame(
        {
            'ID': submission.ID, 
            'target': y_pred_test,
        }
        
    )
    sub.to_csv(f"submission_{model_name}.csv", index=False)
    
    return y_pred_test

In [42]:
test_pred_lightgbm = create_submission(
    model_lgbm, test, "lightgbm4"
)